<a href="https://colab.research.google.com/github/fleithpi/PROGRAMA-O-ORIENTADA-A-OBJETOS/blob/main/biblioteca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# --- 1. Classe Livro (Controle de Estoque e Encapsulamento com Getters/Setters Explícitos) ---
class Livro:
    """
    Representa um livro na biblioteca com controle de cópias.
    Atributos internos são protegidos com '_'.
    """
    def __init__(self, titulo: str, autor: str, copias_totais: int):
        self._titulo = titulo
        self._autor = autor
        self._copias_totais = self._validar_copias(copias_totais)
        self._copias_disponiveis = self._copias_totais

    def _validar_copias(self, quantidade: int) -> int:
        """Método interno para garantir que as cópias são não-negativas."""
        if not isinstance(quantidade, int) or quantidade < 0:
            raise ValueError("O número de cópias deve ser um inteiro não negativo.")
        return quantidade

    # --- Getters Explícitos ---
    def get_titulo(self) -> str:
        return self._titulo

    def get_autor(self) -> str:
        return self._autor

    def get_copias_totais(self) -> int:
        return self._copias_totais

    def get_copias_disponiveis(self) -> int:
        return self._copias_disponiveis

    # --- Setters/Métodos de Consistência (Usados pela Biblioteca) ---
    def aumentar_copias(self, quantidade: int):
        """Aumenta o total e o disponível de forma consistente."""
        quantidade = self._validar_copias(quantidade)
        if quantidade > 0:
            self._copias_totais += quantidade
            self._copias_disponiveis += quantidade
            return True
        return False

    def emprestar(self) -> bool:
        """Decrementa cópias disponíveis, se houver."""
        if self._copias_disponiveis > 0:
            self._copias_disponiveis -= 1
            return True
        return False

    def devolver(self) -> bool:
        """Incrementa cópias disponíveis (garante que não exceda o total)."""
        if self._copias_disponiveis < self._copias_totais:
            self._copias_disponiveis += 1
            return True
        return False

    def __str__(self):
        return (f"'{self._titulo}' por {self._autor} "
                f"({self._copias_disponiveis}/{self._copias_totais} disponíveis)")

# --- 2. Classe Usuario (Controle de Empréstimos e Encapsulamento com Getters Explícitos) ---
class Usuario:
    """
    Representa um usuário com sua lista de livros emprestados.
    """
    def __init__(self, id_usuario: int, nome: str):
        self._id = id_usuario
        self._nome = nome
        self._livros_emprestados: list[Livro] = []

    # --- Getters Explícitos ---
    def get_id(self) -> int:
        return self._id

    def get_nome(self) -> str:
        return self._nome

    def get_livros_emprestados(self) -> list[Livro]:
        """Retorna a lista de objetos Livro emprestados."""
        return self._livros_emprestados

    # --- MÉTODOS DE CONTROLE (Usados pela Biblioteca para manter a consistência) ---
    def adicionar_emprestado(self, livro: Livro):
        """Método para adicionar um objeto Livro à lista do usuário."""
        self._livros_emprestados.append(livro)

    def remover_emprestado(self, livro: Livro) -> bool:
        """Método para remover um objeto Livro da lista do usuário."""
        for i, item in enumerate(self._livros_emprestados):
            if item.get_titulo() == livro.get_titulo():
                del self._livros_emprestados[i]
                return True
        return False

    # --- MÉTODO SOLICITADO (Visualização) ---
    def listar_livros(self):
        """Exibe os livros atualmente emprestados por este usuário."""
        print(f"\n--- Livros Atualmente Emprestados por {self.get_nome()} (ID: {self.get_id()}) ---")
        if self._livros_emprestados:
            for livro in self._livros_emprestados:
                print(f"- {livro.get_titulo()} (Autor: {livro.get_autor()})")
        else:
            print("Nenhum livro emprestado no momento.")

# --- 3. Classe Biblioteca (O Sistema de Gerenciamento) ---
class Biblioteca:
    """
    Gerencia o acervo de Livros e a base de Usuários, coordenando empréstimos e devoluções.
    Todos os dados de estado são encapsulados.
    """
    def __init__(self):
        # Atributos privados
        self._acervo: dict[str, Livro] = {}       # Chave: Título do Livro
        self._usuarios: dict[int, Usuario] = {}   # Chave: ID do Usuário

    # --- Gerenciamento de Livros ---
    def adicionar_livros(self, livro: Livro):
        # Acessa o título usando o getter
        titulo = livro.get_titulo()
        if titulo in self._acervo:
            try:
                # Usa o método de consistência do Livro
                self._acervo[titulo].aumentar_copias(livro.get_copias_totais())
                print(f" Adicionado mais {livro.get_copias_totais()} cópias de '{titulo}'.")
            except ValueError as e:
                print(f" Erro ao adicionar cópias: {e}")
        else:
            self._acervo[titulo] = livro
            print(f" Livro '{titulo}' adicionado ao acervo.")

    # --- Gerenciamento de Usuários ---
    def cadastrar_usuario(self, usuario: Usuario):
        id_usuario = usuario.get_id()
        if id_usuario not in self._usuarios:
            self._usuarios[id_usuario] = usuario
            print(f" Usuário '{usuario.get_nome()}' (ID: {id_usuario}) cadastrado com sucesso.")
        else:
            print(f" Erro: ID de usuário {id_usuario} já existe.")

    # --- MÉTODOS DE OPERAÇÃO (Coordenando Consistência) ---
    def emprestar_livro(self, id_usuario: int, titulo_livro: str):
        usuario = self._usuarios.get(id_usuario)
        livro = self._acervo.get(titulo_livro)

        if not usuario:
            print(f" Erro: Usuário ID {id_usuario} não encontrado.")
            return
        if not livro:
            print(f" Erro: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        # 1. Tentar alterar o estoque (Encapsulamento do Livro)
        if livro.emprestar():
            # 2. Atualizar a lista do Usuário (Encapsulamento do Usuario)
            usuario.adicionar_emprestado(livro)
            print(f" Livro '{titulo_livro}' emprestado com sucesso para {usuario.get_nome()}.")
        else:
            print(f" Livro '{titulo_livro}' não tem cópias disponíveis no momento.")

    def devolver_livro(self, id_usuario: int, titulo_livro: str):
        usuario = self._usuarios.get(id_usuario)
        livro_acervo = self._acervo.get(titulo_livro)

        if not usuario or not livro_acervo:
            print(f" Erro: Usuário ou Livro não encontrado.")
            return

        # 1. Tentar remover o livro da lista do Usuário (Encapsulamento do Usuario)
        if usuario.remover_emprestado(livro_acervo):
            # 2. Se o livro foi removido do usuário, atualizar o estoque (Encapsulamento do Livro)
            livro_acervo.devolver()
            print(f" Livro '{titulo_livro}' devolvido com sucesso por {usuario.get_nome()}.")
        else:
             print(f" Erro: O usuário {usuario.get_nome()} não tinha o livro '{titulo_livro}' emprestado.")

    # --- MÉTODOS DE VISUALIZAÇÃO ---

    def listar_livros_disponiveis(self):
        """Exibe os livros atualmente disponíveis no acervo."""
        print("\n--- Acervo Disponível ---")
        disponiveis = [l for l in self._acervo.values() if l.get_copias_disponiveis() > 0]
        if disponiveis:
            for livro in disponiveis:
                print(f"- {livro}")
        else:
            print("Nenhum livro disponível no momento.")

    def listar_emprestimos_usuario(self, id_usuario: int):
        """Exibe os livros emprestados por um usuário específico."""
        usuario = self._usuarios.get(id_usuario)
        if usuario:
            usuario.listar_livros()
        else:
            print(f" Erro: Usuário ID {id_usuario} não encontrado.")

# --- FOLHA DE EXEMPLO (DEMONSTRAÇÃO DE USO) ---
if __name__ == '__main__':
    print("=========================================")
    print(" INÍCIO DO SISTEMA DE GERENCIAMENTO")
    print("=========================================")

    biblioteca = Biblioteca()

    # 1. Adicionar Livros (com aumento de estoque)
    livro_a = Livro("Padrões de Projeto", "Erich Gamma", 3)
    livro_b = Livro("Clean Code", "Robert C. Martin", 2)
    biblioteca.adicionar_livros(livro_a)
    biblioteca.adicionar_livros(livro_b)

    # Aumentar estoque de Livro A usando um objeto temporário
    # Corrigido o erro percebido anteriormente: O código é funcional, apenas demonstra a recarga.
    livro_a_copia = Livro("Padrões de Projeto", "Erich Gamma", 1)
    biblioteca.adicionar_livros(livro_a_copia) # Agora o estoque é 4


    # 2. Cadastrar Usuários
    user_alice = Usuario(101, "Alice Silva")
    user_bruno = Usuario(102, "Bruno Mendes")
    biblioteca.cadastrar_usuario(user_alice)
    biblioteca.cadastrar_usuario(user_bruno)

    print("\n--- Estado Inicial do Acervo ---")
    biblioteca.listar_livros_disponiveis() # Livro A: 4/4, Livro B: 2/2

    # 3. Realizar Empréstimos
    print("\n--- Empréstimos ---")
    biblioteca.emprestar_livro(101, "Padrões de Projeto") # Alice
    biblioteca.emprestar_livro(102, "Clean Code")         # Bruno
    biblioteca.emprestar_livro(101, "Clean Code")         # Alice (última cópia)

    # Tentativa de empréstimo falha (sem cópias)
    biblioteca.emprestar_livro(102, "Clean Code")         # Bruno

    print("\n--- Estado Após Empréstimos ---")
    biblioteca.listar_livros_disponiveis() # Livro A: 3/4, Livro B: 0/2

    # 4. Listar Empréstimos por Usuário
    biblioteca.listar_emprestimos_usuario(101) # Alice deve ter 2 livros
    biblioteca.listar_emprestimos_usuario(102) # Bruno deve ter 1 livro

    # 5. Devolução
    print("\n--- Devolução ---")
    biblioteca.devolver_livro(101, "Clean Code") # Alice devolve
    biblioteca.devolver_livro(102, "Clean Code") # Bruno tenta emprestar (e falha)
    biblioteca.devolver_livro(102, "Clean Code") # Bruno devolve seu livro

    print("\n--- Estado Final ---")
    biblioteca.listar_livros_disponiveis() # Livro B deve estar 2/2
    biblioteca.listar_emprestimos_usuario(101)
    biblioteca.listar_emprestimos_usuario(102)

    print("\n=========================================")
    print(" FIM DO SISTEMA")
    print("=========================================")

 INÍCIO DO SISTEMA DE GERENCIAMENTO
 Livro 'Padrões de Projeto' adicionado ao acervo.
 Livro 'Clean Code' adicionado ao acervo.
 Adicionado mais 1 cópias de 'Padrões de Projeto'.
 Usuário 'Alice Silva' (ID: 101) cadastrado com sucesso.
 Usuário 'Bruno Mendes' (ID: 102) cadastrado com sucesso.

--- Estado Inicial do Acervo ---

--- Acervo Disponível ---
- 'Padrões de Projeto' por Erich Gamma (4/4 disponíveis)
- 'Clean Code' por Robert C. Martin (2/2 disponíveis)

--- Empréstimos ---
 Livro 'Padrões de Projeto' emprestado com sucesso para Alice Silva.
 Livro 'Clean Code' emprestado com sucesso para Bruno Mendes.
 Livro 'Clean Code' emprestado com sucesso para Alice Silva.
 Livro 'Clean Code' não tem cópias disponíveis no momento.

--- Estado Após Empréstimos ---

--- Acervo Disponível ---
- 'Padrões de Projeto' por Erich Gamma (3/4 disponíveis)

--- Livros Atualmente Emprestados por Alice Silva (ID: 101) ---
- Padrões de Projeto (Autor: Erich Gamma)
- Clean Code (Autor: Robert C. Martin)

In [47]:
# --- 1. Classe Livro (Controle de Estoque e Encapsulamento) ---
class Livro:
    """
    Representa um livro na biblioteca com controle de cópias.
    Atributos internos são protegidos com '_'.
    """
    def __init__(self, titulo: str, autor: str, copias_totais: int):
        self._titulo = titulo
        self._autor = autor
        self._copias_totais = copias_totais
        self._copias_disponiveis = copias_totais

    # --- Getters (Acesso controlado aos dados) ---
    @property
    def titulo(self):
        return self._titulo

    @property
    def autor(self):
        return self._autor

    @property
    def copias_disponiveis(self):
        return self._copias_disponiveis

    # --- Métodos de Consistência (Usados pela Biblioteca) ---
    def aumentar_copias(self, quantidade: int):
        """Aumenta o total e o disponível de forma consistente."""
        if quantidade > 0:
            self._copias_totais += quantidade
            self._copias_disponiveis += quantidade
        else:
            raise ValueError("A quantidade de cópias deve ser positiva.")

    def emprestar(self) -> bool:
        """Decrementa cópias disponíveis, se houver."""
        if self._copias_disponiveis > 0:
            self._copias_disponiveis -= 1
            return True
        return False

    def devolver(self) -> bool:
        """Incrementa cópias disponíveis (garante que não exceda o total)."""
        if self._copias_disponiveis < self._copias_totais:
            self._copias_disponiveis += 1
            return True
        return False

    def __str__(self):
        return (f"'{self._titulo}' por {self._autor} "
                f"({self._copias_disponiveis}/{self._copias_totais} disponíveis)")

# --- 2. Classe Usuario (Controle de Empréstimos e Encapsulamento) ---
class Usuario:
    """
    Representa um usuário com sua lista de livros emprestados.
    """
    def __init__(self, id_usuario: int, nome: str):
        self._id = id_usuario
        self._nome = nome
        # Armazena os OBJETOS Livro emprestados
        self._livros_emprestados: list[Livro] = []

    # --- Getters ---
    @property
    def id(self):
        return self._id

    @property
    def nome(self):
        return self._nome

    # --- MÉTODOS DE CONTROLE (Usados pela Biblioteca para manter a consistência) ---
    def adicionar_emprestado(self, livro: Livro):
        """Método privado para adicionar um objeto Livro à lista do usuário."""
        self._livros_emprestados.append(livro)

    def remover_emprestado(self, livro: Livro) -> bool:
        """Método privado para remover um objeto Livro da lista do usuário."""
        # Procuramos uma instância do livro com o mesmo título (simplificação)
        for i, item in enumerate(self._livros_emprestados):
            if item.titulo == livro.titulo:
                del self._livros_emprestados[i]
                return True
        return False

    # --- MÉTODO SOLICITADO (Visualização) ---
    def listar_livros(self):
        """Exibe os livros atualmente emprestados por este usuário."""
        print(f"\n--- Livros Atualmente Emprestados por {self._nome} (ID: {self._id}) ---")
        if self._livros_emprestados:
            for livro in self._livros_emprestados:
                print(f"- {livro.titulo} (Autor: {livro.autor})")
        else:
            print("Nenhum livro emprestado no momento.")

# --- 3. Classe Biblioteca (O Sistema de Gerenciamento) ---
class Biblioteca:
    """
    Gerencia o acervo de Livros e a base de Usuários, coordenando empréstimos e devoluções.
    Todos os dados de estado são encapsulados.
    """
    def __init__(self):
        # Atributos privados
        self._acervo: dict[str, Livro] = {}       # Chave: Título do Livro
        self._usuarios: dict[int, Usuario] = {}   # Chave: ID do Usuário

    # --- Gerenciamento de Livros ---
    def adicionar_livros(self, livro: Livro):
        if livro.titulo in self._acervo:
            try:
                self._acervo[livro.titulo].aumentar_copias(livro.copias_totais)
                print(f"✅ Adicionado mais {livro.copias_totais} cópias de '{livro.titulo}'.")
            except ValueError as e:
                print(f"❌ Erro ao adicionar cópias: {e}")
        else:
            self._acervo[livro.titulo] = livro
            print(f"✅ Livro '{livro.titulo}' adicionado ao acervo.")

    # --- Gerenciamento de Usuários ---
    def cadastrar_usuario(self, usuario: Usuario):
        if usuario.id not in self._usuarios:
            self._usuarios[usuario.id] = usuario
            print(f"✅ Usuário '{usuario.nome}' (ID: {usuario.id}) cadastrado com sucesso.")
        else:
            print(f"❌ Erro: ID de usuário {usuario.id} já existe.")

    # --- MÉTODOS DE OPERAÇÃO (Coordenando Consistência) ---
    def emprestar_livro(self, id_usuario: int, titulo_livro: str):
        """
        Adiciona um livro à lista de livros emprestados do usuário, se disponível.
        Controla o estoque.
        """
        usuario = self._usuarios.get(id_usuario)
        livro = self._acervo.get(titulo_livro)

        if not usuario:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return
        if not livro:
            print(f"❌ Erro: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        # 1. Tentar alterar o estoque (Encapsulamento do Livro)
        if livro.emprestar():
            # 2. Atualizar a lista do Usuário (Encapsulamento do Usuario)
            usuario.adicionar_emprestado(livro)
            print(f"🎉 Livro '{titulo_livro}' emprestado com sucesso para {usuario.nome}.")
        else:
            print(f"⚠️ Livro '{titulo_livro}' não tem cópias disponíveis no momento.")

    def devolver_livro(self, id_usuario: int, titulo_livro: str):
        """
        Remove o livro da lista de livros emprestados do usuário.
        Atualiza o estoque.
        """
        usuario = self._usuarios.get(id_usuario)
        livro_acervo = self._acervo.get(titulo_livro) # Pega o objeto Livro do acervo para manipular o estoque

        if not usuario or not livro_acervo:
            print(f"❌ Erro: Usuário ou Livro não encontrado.")
            return

        # 1. Tentar remover o livro da lista do Usuário (Encapsulamento do Usuario)
        # Passamos o objeto Livro do acervo para que o usuário possa identificá-lo pelo título
        if usuario.remover_emprestado(livro_acervo):
            # 2. Se o livro foi removido do usuário, atualizar o estoque (Encapsulamento do Livro)
            livro_acervo.devolver()
            print(f"🎉 Livro '{titulo_livro}' devolvido com sucesso por {usuario.nome}.")
        else:
             print(f"❌ Erro: O usuário {usuario.nome} não tinha o livro '{titulo_livro}' emprestado.")

    # --- MÉTODOS DE VISUALIZAÇÃO ---

    def listar_livros_disponiveis(self):
        """Exibe os livros atualmente disponíveis no acervo."""
        print("\n--- Acervo Disponível ---")
        disponiveis = [l for l in self._acervo.values() if l.copias_disponiveis > 0]
        if disponiveis:
            for livro in disponiveis:
                print(f"- {livro}")
        else:
            print("Nenhum livro disponível no momento.")

    def listar_emprestimos_usuario(self, id_usuario: int):
        """Exibe os livros emprestados por um usuário específico."""
        usuario = self._usuarios.get(id_usuario)
        if usuario:
            usuario.listar_livros()
        else:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")

# --- FOLHA DE EXEMPLO (DEMONSTRAÇÃO DE USO) ---
if __name__ == '__main__':
    print("=========================================")
    print(" INÍCIO DO SISTEMA DE GERENCIAMENTO")
    print("=========================================")

    biblioteca = Biblioteca()

    # 1. Adicionar Livros (com aumento de estoque)
    livro_a = Livro("Padrões de Projeto", "Erich Gamma", 3)
    livro_b = Livro("Clean Code", "Robert C. Martin", 2)
    biblioteca.adicionar_livros(livro_a)
    biblioteca.adicionar_livros(livro_b)

    # Aumentar estoque de Livro A
    livro_a_copia = Livro("Padrões de Projeto", "Erich Gamma", 1)
    biblioteca.adicionar_livros(livro_a_copia)

    # 2. Cadastrar Usuários
    user_alice = Usuario(101, "Alice Silva")
    user_bruno = Usuario(102, "Bruno Mendes")
    biblioteca.cadastrar_usuario(user_alice)
    biblioteca.cadastrar_usuario(user_bruno)

    print("\n--- Estado Inicial do Acervo ---")
    biblioteca.listar_livros_disponiveis() # Livro A: 4/4, Livro B: 2/2

    # 3. Realizar Empréstimos
    print("\n--- Empréstimos ---")
    biblioteca.emprestar_livro(101, "Padrões de Projeto") # Alice
    biblioteca.emprestar_livro(102, "Clean Code")         # Bruno
    biblioteca.emprestar_livro(101, "Clean Code")         # Alice (última cópia)

    # Tentativa de empréstimo falha (sem cópias)
    biblioteca.emprestar_livro(102, "Clean Code")         # Bruno

    print("\n--- Estado Após Empréstimos ---")
    biblioteca.listar_livros_disponiveis() # Livro A: 3/4, Livro B: 0/2

    # 4. Listar Empréstimos por Usuário
    biblioteca.listar_emprestimos_usuario(101) # Alice deve ter 2 livros
    biblioteca.listar_emprestimos_usuario(102) # Bruno deve ter 1 livro

    # 5. Devolução
    print("\n--- Devolução ---")
    biblioteca.devolver_livro(101, "Clean Code") # Alice devolve
    biblioteca.devolver_livro(102, "Clean Code") # Bruno tenta devolver o que não tem mais (falha)
    biblioteca.devolver_livro(102, "Clean Code") # Bruno devolve seu livro

    print("\n--- Estado Final ---")
    biblioteca.listar_livros_disponiveis() # Livro B deve estar 2/2
    biblioteca.listar_emprestimos_usuario(101)
    biblioteca.listar_emprestimos_usuario(102)

    print("\n=========================================")
    print(" FIM DO SISTEMA")
    print("=========================================")

 INÍCIO DO SISTEMA DE GERENCIAMENTO
✅ Livro 'Padrões de Projeto' adicionado ao acervo.
✅ Livro 'Clean Code' adicionado ao acervo.


AttributeError: 'Livro' object has no attribute 'copias_totais'

In [39]:
class Livro:
    """
    Representa um livro. O acesso aos atributos é feito via @property.
    """
    def __init__(self, titulo: str, autor: str, copias_totais: int):
        self._titulo = titulo
        self._autor = autor
        self._copias_totais = copias_totais
        self._copias_disponiveis = copias_totais

    @property
    def titulo(self):
        return self._titulo

    @property
    def autor(self):
        return self._autor

    @property
    def copias_totais(self):
        return self._copias_totais

    @property
    def copias_disponiveis(self):
        return self._copias_disponiveis

    def aumentar_copias(self, quantidade: int):
        if quantidade > 0:
            self._copias_totais += quantidade
            self._copias_disponiveis += quantidade
        else:
            raise ValueError("A quantidade de cópias deve ser positiva.")

    def emprestar(self):
        if self._copias_disponiveis > 0:
            self._copias_disponiveis -= 1
            return True
        return False

    def devolver(self):
        if self._copias_disponiveis < self._copias_totais:
            self._copias_disponiveis += 1
            return True
        return False

    def __str__(self):
        return (f"'{self._titulo}' por {self._autor} "
                f"({self._copias_disponiveis}/{self._copias_totais} disponíveis)")


class Usuario:
    """
    Representa um usuário. Refatorada para incluir os métodos de empréstimo.
    """
    def __init__(self, id_usuario: int, nome: str):
        self._id = id_usuario
        self._nome = nome
        # Armazena os OBJETOS Livro emprestados para rastreamento
        self._livros_emprestados: list[Livro] = []

    @property
    def id(self):
        return self._id

    @property
    def nome(self):
        return self._nome

    # Método de controle: Usado pela Biblioteca
    def _adicionar_livro_emprestado(self, livro: Livro):
        """Adiciona um objeto Livro à lista de empréstimos do usuário."""
        self._livros_emprestados.append(livro)

    # Método de controle: Usado pela Biblioteca
    def _remover_livro_emprestado(self, livro: Livro) -> bool:
        """Remove um objeto Livro da lista de empréstimos do usuário."""
        try:
            self._livros_emprestados.remove(livro)
            return True
        except ValueError:
            return False

    # --- MÉTODOS SOLICITADOS (para demonstração) ---

    def listar_livros(self):
        """Exibe os livros atualmente emprestados por este usuário."""
        print(f"\n--- Livros Atualmente Emprestados por {self._nome} (ID: {self._id}) ---")
        if self._livros_emprestados:
            for livro in self._livros_emprestados:
                # Aqui exibimos o TÍTULO para o usuário, mas internamente controlamos o objeto
                print(f"- {livro.titulo} (Autor: {livro.autor})")
        else:
            print("Nenhum livro emprestado no momento.")

    # NOTA: Os métodos emprestar_livro(livro) e devolver_livro(livro) no contexto
    # do sistema de biblioteca são na verdade métodos da Biblioteca que interagem
    # com o usuário e o livro. No entanto, se o objetivo é *simplesmente*
    # adicionar/remover da lista interna do usuário, podemos definir assim:

    def emprestar_livro_local(self, livro: Livro) -> None:
        """Adiciona um livro à lista interna (simula empréstimo local)."""
        self._livros_emprestados.append(livro)

    def devolver_livro_local(self, livro: Livro) -> bool:
        """Remove o livro da lista interna (simula devolução local)."""
        return self._remover_livro_emprestado(livro)

In [40]:
class Biblioteca:
    def __init__(self):
        # Atributos privados
        self._acervo: dict[str, Livro] = {}
        self._usuarios: dict[int, Usuario] = {}

    def adicionar_livros(self, livro: Livro):
        # ... (implementação omitida, é a mesma da resposta anterior)
        if livro.titulo in self._acervo:
            try:
                self._acervo[livro.titulo].aumentar_copias(livro.copias_totais)
                print(f"✅ Adicionado mais {livro.copias_totais} cópias de '{livro.titulo}'.")
            except ValueError as e:
                print(f"❌ Erro ao adicionar cópias: {e}")
        else:
            self._acervo[livro.titulo] = livro
            print(f"✅ Livro '{livro.titulo}' adicionado ao acervo.")

    def cadastrar_usuario(self, usuario: Usuario):
        # ... (implementação omitida, é a mesma da resposta anterior)
        if usuario.id not in self._usuarios:
            self._usuarios[usuario.id] = usuario
            print(f"✅ Usuário '{usuario.nome}' (ID: {usuario.id}) cadastrado com sucesso.")
        else:
            print(f"❌ Erro: ID de usuário {usuario.id} já existe.")

    # --- Métodos Coordenadores de Empréstimo e Devolução ---

    def emprestar_livro(self, id_usuario: int, titulo_livro: str):
        """Coordena o empréstimo: afeta estoque do Livro e lista do Usuario."""
        if id_usuario not in self._usuarios or titulo_livro not in self._acervo:
            print(f"❌ Erro: Usuário ou Livro não encontrado.")
            return

        livro = self._acervo[titulo_livro]
        usuario = self._usuarios[id_usuario]

        # 1. Tentar alterar o estoque do Livro
        if livro.emprestar():
            # 2. Se o estoque foi alterado, atualizar a lista do Usuário
            usuario._adicionar_livro_emprestado(livro)
            # Nota: O método _adicionar_livro_emprestado é 'privado' (conforme convenção),
            # mas é usado aqui pela Biblioteca (seu controlador).
            print(f"🎉 Livro '{titulo_livro}' emprestado com sucesso para {usuario.nome}.")
        else:
            print(f"⚠️ Livro '{titulo_livro}' não tem cópias disponíveis no momento.")

    def devolver_livro(self, id_usuario: int, titulo_livro: str):
        """Coordena a devolução: afeta lista do Usuario e estoque do Livro."""
        if id_usuario not in self._usuarios or titulo_livro not in self._acervo:
            print(f"❌ Erro: Usuário ou Livro não encontrado.")
            return

        livro = self._acervo[titulo_livro]
        usuario = self._usuarios[id_usuario]

        # 1. Tentar remover o livro da lista do Usuário
        if usuario._remover_livro_emprestado(livro):
            # 2. Se o livro foi removido do usuário, atualizar o estoque do Livro
            livro.devolver()
            print(f"🎉 Livro '{titulo_livro}' devolvido com sucesso por {usuario.nome}.")
        else:
             print(f"❌ Erro: O usuário {usuario.nome} não tinha o livro '{titulo_livro}' emprestado.")

    # --- Método de Visualização ---

    def listar_livros_disponiveis(self):
        # ... (implementação omitida, é a mesma da resposta anterior)
        print("\n--- Acervo Disponível ---")
        disponiveis = [l for l in self._acervo.values() if l.copias_disponiveis > 0]
        if disponiveis:
            for livro in disponiveis:
                print(f"- {livro}")
        else:
            print("Nenhum livro disponível no momento.")

    def listar_emprestimos_usuario(self, id_usuario: int):
        """Chama o método de visualização do próprio objeto Usuario."""
        if id_usuario in self._usuarios:
            self._usuarios[id_usuario].listar_livros()
        else:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")

In [42]:
biblioteca = Biblioteca()

    # 1. Adicionar Livros (com aumento de estoque)
    livro_a = Livro("Padrões de Projeto", "Erich Gamma", 3)
    livro_b = Livro("Clean Code", "Robert C. Martin", 2)
    biblioteca.adicionar_livros(livro_a)
    biblioteca.adicionar_livros(livro_b)

    # Aumentar estoque de Livro A
    livro_a_copia = Livro("Padrões de Projeto", "Erich Gamma", 1)
    biblioteca.adicionar_livros(livro_a_copia)

    # 2. Cadastrar Usuários
    user_alice = Usuario(101, "Alice Silva")
    user_bruno = Usuario(102, "Bruno Mendes")
    biblioteca.cadastrar_usuario(user_alice)
    biblioteca.cadastrar_usuario(user_bruno)

    print("\n--- Estado Inicial do Acervo ---")
    biblioteca.listar_livros_disponiveis() # Livro A: 4/4, Livro B: 2/2

    # 3. Realizar Empréstimos
    print("\n--- Empréstimos ---")
    biblioteca.emprestar_livro(101, "Padrões de Projeto") # Alice
    biblioteca.emprestar_livro(102, "Clean Code")         # Bruno
    biblioteca.emprestar_livro(101, "Clean Code")         # Alice (última cópia)

    # Tentativa de empréstimo falha (sem cópias)
    biblioteca.emprestar_livro(102, "Clean Code")         # Bruno

    print("\n--- Estado Após Empréstimos ---")
    biblioteca.listar_livros_disponiveis() # Livro A: 3/4, Livro B: 0/2

    # 4. Listar Empréstimos por Usuário
    biblioteca.listar_emprestimos_usuario(101) # Alice deve ter 2 livros
    biblioteca.listar_emprestimos_usuario(102) # Bruno deve ter 1 livro

    # 5. Devolução
    print("\n--- Devolução ---")
    biblioteca.devolver_livro(101, "Clean Code") # Alice devolve
    biblioteca.devolver_livro(102, "Clean Code") # Bruno tenta devolver o que não tem mais (falha)
    biblioteca.devolver_livro(102, "Clean Code") # Bruno devolve seu livro

    print("\n--- Estado Final ---")
    biblioteca.listar_livros_disponiveis() # Livro B deve estar 2/2
    biblioteca.listar_emprestimos_usuario(101)
    biblioteca.listar_emprestimos_usuario(102)

IndentationError: unexpected indent (ipython-input-3665238450.py, line 4)

In [37]:
class Livro:
    """
    Representa um livro com encapsulamento.
    A identificação única é o título (simplificado para este exemplo).
    """
    def __init__(self, titulo: str, autor: str, copias_totais: int):
        self._titulo = titulo
        self._autor = autor
        # Atributos protegidos (encapsulamento)
        self._copias_totais = copias_totais
        self._copias_disponiveis = copias_totais

    # --- Getters (Propriedades para acessar dados protegidos) ---
    @property
    def titulo(self):
        return self._titulo

    @property
    def autor(self):
        return self._autor

    @property
    def copias_totais(self):
        return self._copias_totais

    @property
    def copias_disponiveis(self):
        return self._copias_disponiveis

    # --- Métodos para alteração controlada pelo sistema ---
    def aumentar_copias(self, quantidade: int):
        """Método seguro para aumentar o estoque do livro."""
        if quantidade > 0:
            self._copias_totais += quantidade
            self._copias_disponiveis += quantidade
        else:
            raise ValueError("A quantidade de cópias deve ser positiva.")

    def emprestar(self):
        """Decrementa cópias disponíveis."""
        if self._copias_disponiveis > 0:
            self._copias_disponiveis -= 1
            return True
        return False

    def devolver(self):
        """Incrementa cópias disponíveis (garante que não exceda o total)."""
        if self._copias_disponiveis < self._copias_totais:
            self._copias_disponiveis += 1
            return True
        return False

    def __str__(self):
        return (f"'{self._titulo}' por {self._autor} "
                f"({self._copias_disponiveis}/{self._copias_totais} disponíveis)")


class Usuario:
    """Representa um usuário com encapsulamento."""
    def __init__(self, id_usuario: int, nome: str):
        self._id = id_usuario
        self._nome = nome
        # Lista de títulos de livros emprestados (simples, sem Empréstimo Objeto)
        self._livros_emprestados = []

    # --- Getters ---
    @property
    def id(self):
        return self._id

    @property
    def nome(self):
        return self._nome

    @property
    def livros_emprestados(self):
        return self._livros_emprestados

    # --- Métodos de Consistência (usados pela Biblioteca) ---
    def adicionar_emprestimo(self, titulo: str):
        self._livros_emprestados.append(titulo)

    def remover_emprestimo(self, titulo: str) -> bool:
        if titulo in self._livros_emprestados:
            self._livros_emprestados.remove(titulo)
            return True
        return False

    def __str__(self):
        return f"Usuário {self._nome} (ID: {self._id})"

In [38]:
class Biblioteca:
    # Corrigido o nome de Biclioteca para Biblioteca
    def __init__(self):
        # ATRIBUTOS PRIVADOS (Encapsulamento por Convenção em Python)
        # Usamos o título como chave do acervo para este modelo simplificado
        self._acervo: dict[str, Livro] = {}
        self._usuarios: dict[int, Usuario] = {}

    # --- Métodos de Gerenciamento de Acervo ---

    def adicionar_livros(self, livro: Livro):
        if livro.titulo in self._acervo:
            # Se o livro existe, aumentamos as cópias de forma controlada
            try:
                # Chama o método protegido da classe Livro
                self._acervo[livro.titulo].aumentar_copias(livro.copias_totais)
                print(f"✅ Adicionado mais {livro.copias_totais} cópias de '{livro.titulo}'.")
            except ValueError as e:
                print(f"❌ Erro ao adicionar cópias: {e}")
        else:
            # Se o livro é novo
            self._acervo[livro.titulo] = livro
            print(f"✅ Livro '{livro.titulo}' adicionado ao acervo.")

    # --- Métodos de Gerenciamento de Usuários ---

    def cadastrar_usuario(self, usuario: Usuario):
        if usuario.id not in self._usuarios:
            self._usuarios[usuario.id] = usuario
            print(f"✅ Usuário '{usuario.nome}' (ID: {usuario.id}) cadastrado com sucesso.")
        else:
            print(f"❌ Erro: ID de usuário {usuario.id} já existe.")

    # --- Métodos de Operação (Empréstimo/Devolução) ---

    def emprestar_livro(self, id_usuario: int, titulo_livro: str):
        # 1. Verificar se o usuário existe (Acessando o atributo privado _usuarios)
        if id_usuario not in self._usuarios:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return

        # 2. Verificar se o livro existe (Acessando o atributo privado _acervo)
        if titulo_livro not in self._acervo:
            print(f"❌ Erro: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        livro = self._acervo[titulo_livro]
        usuario = self._usuarios[id_usuario]

        # 3. Tentar realizar o empréstimo (usa o método encapsulado do Livro)
        if livro.emprestar():
            # 4. Atualizar registro do Usuário (usa o método encapsulado do Usuário)
            usuario.adicionar_emprestimo(livro.titulo)
            print(f"🎉 Livro '{titulo_livro}' emprestado com sucesso para {usuario.nome}.")
        else:
            print(f"⚠️ Livro '{titulo_livro}' não tem cópias disponíveis no momento.")

    def devolver_livro(self, id_usuario: int, titulo_livro: str):
        # 1. Verificar se o usuário existe
        if id_usuario not in self._usuarios:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return

        usuario = self._usuarios[id_usuario]

        # 2. Verificar se o usuário realmente tem o livro emprestado
        if titulo_livro not in usuario.livros_emprestados:
            print(f"❌ Erro: O usuário {usuario.nome} não tem o livro '{titulo_livro}' emprestado.")
            return

        # 3. Realizar a devolução (Remove do Usuário e adiciona no Livro)
        usuario.remover_emprestimo(titulo_livro)

        if titulo_livro in self._acervo:
            self._acervo[titulo_livro].devolver() # Usa o método encapsulado do Livro
            print(f"🎉 Livro '{titulo_livro}' devolvido com sucesso por {usuario.nome}.")
        else:
             # Isso só deve ocorrer se houver um erro de lógica
             print(f"⚠️ Livro devolvido, mas não encontrado no acervo (aviso).")

    # --- Métodos de Visualização ---

    def listar_livros_disponiveis(self):
        print("\n--- Acervo Disponível ---")
        # Itera sobre os objetos Livro no dicionário privado _acervo
        disponiveis = [l for l in self._acervo.values() if l.copias_disponiveis > 0]
        if disponiveis:
            for livro in disponiveis:
                print(f"- {livro}")
        else:
            print("Nenhum livro disponível no momento.")

    def listar_emprestimos_usuario(self, id_usuario: int):
        if id_usuario in self._usuarios:
            usuario = self._usuarios[id_usuario]
            print(f"\n--- Livros Emprestados por {usuario.nome} (ID: {id_usuario}) ---")
            if usuario.livros_emprestados:
                for titulo in usuario.livros_emprestados:
                    print(f"- {titulo}")
            else:
                print("Nenhum livro emprestado.")
        else:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")

# --- As classes Livro e Usuario devem permanecer como foram definidas anteriormente ---
# (Elas já usavam atributos privados como _titulo, _id, _livros_emprestados)
# ... (Código das classes Livro e Usuario omitido para concisão)

In [28]:
from __future__ import annotations
from typing import List, Optional

# ======================================================================
# CLASSE LIVRO
# Representa um livro com seu estado de disponibilidade.
# ======================================================================
class Livro:
    """Representa um livro no acervo da biblioteca."""
    def __init__(self, titulo: str, autor: str):
        """
        Inicializa um objeto Livro.
        Por padrão, um novo livro sempre está disponível.
        """
        self.titulo = titulo
        self.autor = autor
        self.disponivel = True  # Atributo para indicar disponibilidade

    def emprestar(self) -> bool:
        """
        Tenta emprestar o livro.
        Atualiza o status para indisponível se o livro estiver disponível.
        Retorna True se o empréstimo foi bem-sucedido, False caso contrário.
        """
        if self.disponivel:
            self.disponivel = False
            print(f"INFO: O livro '{self.titulo}' foi marcado como indisponível.")
            return True
        else:
            print(f"AVISO: O livro '{self.titulo}' já está emprestado.")
            return False

    def devolver(self):
        """
        Atualiza o status do livro para disponível.
        """
        self.disponivel = True
        print(f"INFO: O livro '{self.titulo}' foi marcado como disponível.")

    def __str__(self) -> str:
        """Retorna uma representação em string do livro para fácil visualização."""
        status = "Disponível" if self.disponivel else "Emprestado"
        return f"'{self.titulo}' por {self.autor} (Status: {status})"

# ======================================================================
# CLASSE USUARIO
# Representa um membro da biblioteca e os livros que ele pegou.
# ======================================================================
class Usuario:
    """Representa um usuário (membro) da biblioteca."""
    def __init__(self, nome: str, id_usuario: int):
        """
        Inicializa um objeto Usuario.

        Args:
            nome (str): O nome do usuário.
            id_usuario (int): Um ID único para identificar o usuário.
        """
        self.nome = nome
        self.id = id_usuario
        # A lista armazena os objetos Livro completos
        self.livros_emprestados: List[Livro] = []

    def emprestar_livro(self, livro: Livro):
        """Adiciona um objeto Livro à lista de empréstimos do usuário."""
        if livro not in self.livros_emprestados:
            self.livros_emprestados.append(livro)
            print(f"INFO: '{livro.titulo}' adicionado à lista de {self.nome}.")

    def devolver_livro(self, livro: Livro):
        """Remove um objeto Livro da lista de empréstimos do usuário."""
        if livro in self.livros_emprestados:
            self.livros_emprestados.remove(livro)
            print(f"INFO: '{livro.titulo}' removido da lista de {self.nome}.")
        else:
            print(f"AVISO: {self.nome} não pode devolver '{livro.titulo}' pois não o possui.")

    def listar_livros(self):
        """Exibe os títulos dos livros que o usuário pegou emprestado."""
        print(f"\n--- Livros com {self.nome} ---")
        if not self.livros_emprestados:
            print("Nenhum livro emprestado no momento.")
        else:
            for livro in self.livros_emprestados:
                print(f"- {livro.titulo} por {livro.autor}")

    def __str__(self) -> str:
        return f"Usuário: {self.nome} (ID: {self.id})"

# ======================================================================
# CLASSE BIBLIOTECA
# Orquestra as interações entre livros e usuários.
# ======================================================================
class Biblioteca:
    """Gerencia o acervo de livros e o cadastro de usuários."""
    def __init__(self):
        self.livros: List[Livro] = []      # Lista de objetos Livro
        self.usuarios: List[Usuario] = []  # Lista de objetos Usuario

    # --- Métodos de Cadastro ---
    def adicionar_livro(self, livro: Livro):
        """Adiciona um novo livro ao acervo da biblioteca."""
        self.livros.append(livro)
        print(f"✅ Livro '{livro.titulo}' adicionado ao acervo da biblioteca.")

    def cadastrar_usuario(self, usuario: Usuario):
        """Cadastra um novo usuário na biblioteca."""
        self.usuarios.append(usuario)
        print(f"✅ Usuário '{usuario.nome}' cadastrado.")

    # --- Métodos de Busca (Auxiliares) ---
    def buscar_livro_por_titulo(self, titulo: str) -> Optional[Livro]:
        """Encontra e retorna um objeto Livro pelo seu título."""
        for livro in self.livros:
            if livro.titulo == titulo:
                return livro
        return None # Retorna None se não encontrar

    def buscar_usuario_por_id(self, id_usuario: int) -> Optional[Usuario]:
        """Encontra e retorna um objeto Usuario pelo seu ID."""
        for usuario in self.usuarios:
            if usuario.id == id_usuario:
                return usuario
        return None

    # --- Métodos de Operação ---
    def realizar_emprestimo(self, id_usuario: int, titulo_livro: str):
        """Orquestra o processo completo de empréstimo de um livro."""
        print(f"\n>> Tentativa de empréstimo: '{titulo_livro}' para usuário ID {id_usuario}...")

        usuario = self.buscar_usuario_por_id(id_usuario)
        livro = self.buscar_livro_por_titulo(titulo_livro)

        if not usuario:
            print(f" ERRO: Usuário com ID {id_usuario} não encontrado.")
            return
        if not livro:
            print(f" ERRO: Livro com título '{titulo_livro}' não encontrado.")
            return

        # 1. Tenta mudar o status do livro para indisponível
        if livro.emprestar():
            # 2. Se for bem-sucedido, adiciona o livro à lista do usuário
            usuario.emprestar_livro(livro)
            print(f" SUCESSO! Empréstimo de '{titulo_livro}' para {usuario.nome} realizado.")
        else:
            print(f" FALHA! Não foi possível realizar o empréstimo.")

    def realizar_devolucao(self, id_usuario: int, titulo_livro: str):
        """Orquestra o processo completo de devolução de um livro."""
        print(f"\n>> Tentativa de devolução: '{titulo_livro}' pelo usuário ID {id_usuario}...")

        usuario = self.buscar_usuario_por_id(id_usuario)
        livro = self.buscar_livro_por_titulo(titulo_livro)

        if not usuario:
            print(f" ERRO: Usuário com ID {id_usuario} não encontrado.")
            return
        if not livro:
            print(f" ERRO: Livro com título '{titulo_livro}' não encontrado.")
            return

        # 1. Verifica se o livro está na lista de livros do usuário
        if livro in usuario.livros_emprestados:
            # 2. Muda o status do livro para disponível
            livro.devolver()
            # 3. Remove o livro da lista do usuário
            usuario.devolver_livro(livro)
            print(f"
            print(f"FALHA! O usuário {usuario.nome} não possui este livro para devolver.")

    def listar_acervo_completo(self):
        """Exibe todos os livros da biblioteca e seus status."""
        print("\n--- ACERVO COMPLETO DA BIBLIOTECA ---")
        if not self.livros:
            print("Nenhum livro no acervo.")
        for livro in self.livros:
            print(f"- {livro}")

# ======================================================================
# SIMULAÇÃO DO SISTEMA
# ======================================================================
if __name__ == "__main__":
    # 1. Criar a biblioteca
    biblioteca_central = Biblioteca()

    # 2. Adicionar livros ao acervo
    biblioteca_central.adicionar_livro(Livro("A Arte da Guerra", "Sun Tzu"))
    biblioteca_central.adicionar_livro(Livro("O Príncipe", "Maquiavel"))
    biblioteca_central.adicionar_livro(Livro("1984", "George Orwell"))

    # 3. Cadastrar usuários
    biblioteca_central.cadastrar_usuario(Usuario("Carlos", 101))
    biblioteca_central.cadastrar_usuario(Usuario("Beatriz", 102))

    # 4. Exibir o estado inicial do acervo
    biblioteca_central.listar_acervo_completo()

    # 5. Realizar operações de empréstimo
    biblioteca_central.realizar_emprestimo(101, "1984") # Empréstimo válido
    biblioteca_central.realizar_emprestimo(102, "1984") # Deve falhar, livro já emprestado
    biblioteca_central.realizar_emprestimo(102, "A Arte da Guerra") # Empréstimo válido

    # 6. Exibir o estado atual do acervo e dos usuários
    biblioteca_central.listar_acervo_completo()

    carlos = biblioteca_central.buscar_usuario_por_id(101)
    if carlos:
        carlos.listar_livros()

    beatriz = biblioteca_central.buscar_usuario_por_id(102)
    if beatriz:
        beatriz.listar_livros()

    # 7. Realizar operações de devolução
    biblioteca_central.realizar_devolucao(101, "O Príncipe") # Deve falhar, Carlos não tem esse livro
    biblioteca_central.realizar_devolucao(101, "1984") # Devolução válida

    # 8. Exibir estado final
    biblioteca_central.listar_acervo_completo()
    if carlos:
        carlos.listar_livros()

SyntaxError: unterminated f-string literal (detected at line 169) (ipython-input-2161406046.py, line 169)

In [23]:
class Livro:
    def __init__(self, titulo, autor):
        self.__titulo = titulo
        self.__autor = autor
        self.__disponivel = True

    def get_titulo(self):
        return self.__titulo

    def emprestar(self):
        if self.__disponivel:
            self.__disponivel = False
            # LINHA CORRIGIDA AQUI
            print(f" O livro '{self.get_titulo()}' foi emprestado com sucesso!")
            return True
        else:
            print(f" O livro '{self.get_titulo()}' já está emprestado.")
            return False

# Teste
livro_teste = Livro("Padrões de Projeto", "GoF")
livro_teste.emprestar()
livro_teste.emprestar() # Tentando emprestar novamente

 O livro 'Padrões de Projeto' foi emprestado com sucesso!
 O livro 'Padrões de Projeto' já está emprestado.


False

In [24]:
class Usuario:
    def __init__(self, nome: str, id_usuario: int):
        self.nome = nome
        self.id = id_usuario
        self.livros_emprestados = []  # Armazena os TÍTULOS dos livros

    def __str__(self):
        return f"ID: {self.id} | Nome: {self.nome}"

In [26]:
class Biblioteca:
    def __init__(self):
        # Dicionários para acesso rápido por chave
        self.acervo = {}    # {titulo: Livro_objeto}
        self.usuarios = {}  # {id_usuario: Usuario_objeto}

    # --- Gerenciamento de Cadastro ---

    def adicionar_livro(self, livro: Livro):
        if livro.titulo in self.acervo:
            print(f"⚠️ Livro '{livro.titulo}' já está no acervo (considerando uma cópia única por objeto).")
        else:
            self.acervo[livro.titulo] = livro
            print(f"✅ Livro '{livro.titulo}' adicionado ao acervo.")

    def cadastrar_usuario(self, usuario: Usuario):
        if usuario.id not in self.usuarios:
            self.usuarios[usuario.id] = usuario
            print(f"✅ Usuário '{usuario.nome}' cadastrado com ID: {usuario.id}.")
        else:
            print(f"❌ Erro: ID de usuário {usuario.id} já existe.")

    # --- Operações Principais ---

    def emprestar_livro(self, id_usuario: int, titulo_livro: str):
        # 1. Validação
        if id_usuario not in self.usuarios:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return
        if titulo_livro not in self.acervo:
            print(f"❌ Erro: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        livro = self.acervo[titulo_livro]
        usuario = self.usuarios[id_usuario]

        # 2. Execução (chama o método encapsulado do Livro)
        if livro.emprestar():
            usuario.livros_emprestados.append(livro.titulo)
            print(f"🎉 SUCESSO! '{livro.titulo}' emprestado para {usuario.nome}.")
        else:
            print(f"⚠️ FALHA! '{livro.titulo}' está atualmente indisponível.")

    def devolver_livro(self, id_usuario: int, titulo_livro: str):
        # 1. Validação
        if id_usuario not in self.usuarios:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return

        usuario = self.usuarios[id_usuario]

        if titulo_livro not in usuario.livros_emprestados:
            print(f"❌ Erro: O usuário {usuario.nome} não possui o livro '{titulo_livro}'.")
            return

        # 2. Execução (chama o método encapsulado do Livro)
        if titulo_livro in self.acervo:
            livro = self.acervo[titulo_livro]
            livro.devolver()
            usuario.livros_emprestados.remove(titulo_livro)
            print(f"🎉 SUCESSO! '{titulo_livro}' devolvido por {usuario.nome}.")
        else:
             print(f"⚠️ Livro devolvido, mas não encontrado no acervo (erro de sistema).")

    # --- Visualização ---

    def listar_acervo(self):
        print("\n--- ACERVO COMPLETO ---")
        if not self.acervo:
            print("Acervo vazio.")
            return

        for livro in self.acervo.values():
            print(f"- {livro}")

    def listar_emprestimos_usuario(self, id_usuario: int):
        if id_usuario in self.usuarios:
            usuario = self.usuarios[id_usuario]
            print(f"\n--- Livros Emprestados por {usuario.nome} ---")
            if usuario.livros_emprestados:
                for titulo in usuario.livros_emprestados:
                    print(f"- {titulo}")
            else:
                print("Nenhum livro atualmente emprestado.")
        else:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")

# ======================================================================
# SIMULAÇÃO DO SISTEMA
# ======================================================================

minha_biblioteca = Biblioteca()

# Cadastro Inicial
livro1 = Livro("Python Fluente", "Luciano Ramalho")
livro2 = Livro("O Programador Pragmático", "Andrew Hunt")
livro3 = Livro("Estruturas de Dados", "Cormen et al")

minha_biblioteca.adicionar_livro(livro1)
minha_biblioteca.adicionar_livro(livro2)
minha_biblioteca.adicionar_livro(livro3)

alice = Usuario("Alice Silva", 10)
bob = Usuario("Bob Santos", 20)

minha_biblioteca.cadastrar_usuario(alice)
minha_biblioteca.cadastrar_usuario(bob)

minha_biblioteca.listar_acervo()

# --- Operações de Empréstimo ---
print("\n" + "="*40 + "\nEMPRÉSTIMOS\n" + "="*40)

# Alice empresta dois livros
minha_biblioteca.emprestar_livro(10, "Python Fluente")
minha_biblioteca.emprestar_livro(10, "O Programador Pragmático")

# Bob tenta emprestar um livro que Alice já pegou (deve falhar)
minha_biblioteca.emprestar_livro(20, "Python Fluente")

# Bob empresta um livro disponível
minha_biblioteca.emprestar_livro(20, "Estruturas de Dados")

minha_biblioteca.listar_acervo()
minha_biblioteca.listar_emprestimos_usuario(10)

# --- Operações de Devolução ---
print("\n" + "="*40 + "\nDEVOLUÇÕES\n" + "="*40)

# Alice devolve um livro
minha_biblioteca.devolver_livro(10, "Python Fluente")

# Bob tenta devolver um livro que não pegou
minha_biblioteca.devolver_livro(20, "O Programador Pragmático")

minha_biblioteca.listar_acervo()
minha_biblioteca.listar_emprestimos_usuario(10)

AttributeError: 'Livro' object has no attribute 'titulo'

In [20]:
class Livro:
    """
    Representa um livro com título e autor encapsulados,
    e o status de disponibilidade como booleano privado.
    """
    def __init__(self, titulo: str, autor: str):
        # Atributos privados (usando __ para "name mangling")
        self.__titulo = titulo
        self.__autor = autor
        self.__disponivel = True  # Atributo booleano privado

    # --- Getters para Título e Autor ---

    def get_titulo(self) -> str:
        """Retorna o título do livro (acesso controlado)."""
        return self.__titulo

    def get_autor(self) -> str:
        """Retorna o autor do livro (acesso controlado)."""
        return self.__autor

    # --- Métodos de Status/Comportamento ---

    def is_disponivel(self) -> bool:
        """Verifica e retorna se o livro está disponível."""
        return self.__disponivel

    def emprestar(self) -> bool:
        """
        Tenta emprestar o livro.
        Retorna True se o status foi alterado (emprestado), False se já estava emprestado.
        """
        if self.__disponivel:
            self.__disponivel = False
            print("f O livro  {self.titulo} foi emprestado com sucesso
            return True
        return False

    def devolver(self):
        """Marca o livro como disponível novamente."""
        self.__disponivel = True

    # --- Representação do Objeto ---

    def __str__(self):
        status = "Disponível" if self.__disponivel else "Emprestado"
        return f"'{self.__titulo}' por {self.__autor} [Status: {status}]"

# ----------------------------------------------------------------------
# EXEMPLO DE USO
# ----------------------------------------------------------------------
livro_exemplo = Livro("Estruturas de Dados em Python", "Maria da Silva")

print(livro_exemplo) # Status: Disponível

# Empréstimo
livro_exemplo.emprestar()
print(f"Título: {livro_exemplo.get_titulo()} | Disponível? {livro_exemplo.is_disponivel()}")

# Tentativa de acesso direto (não recomendado e difícil em Python)
# print(livro_exemplo.__titulo) # Isso geraria um AttributeError

# Devolução
livro_exemplo.devolver()
print(livro_exemplo) # Status: Disponível

'Estruturas de Dados em Python' por Maria da Silva [Status: Disponível]
f O livro'(self.titulo)' foi emprestado com sucesso.
Título: Estruturas de Dados em Python | Disponível? False
'Estruturas de Dados em Python' por Maria da Silva [Status: Disponível]


In [16]:
# --- CLASSE LIVRO (com atributos privados e métodos de encapsulamento) ---
class Livro:
    def __init__(self, titulo, autor):
        self.__titulo = titulo
        self.__autor = autor
        self.__disponivel = True
        # Usamos atributos publicos para compatibilidade com o dicionário da Biblioteca
        self.titulo = titulo
        self.copias_totais = 1 # Cada objeto é uma única cópia
        self.copias_disponiveis = 1 # Para simular disponibilidade única

    def emprestar(self):
        if self.__disponivel:
            self.__disponivel = False
            self.copias_disponiveis = 0 # Atualiza o simulador de cópias
            return True
        return False

    def devolver(self):
        self.__disponivel = True
        self.copias_disponiveis = 1 # Atualiza o simulador de cópias

    def is_disponivel(self):
        return self.__disponivel

    def __str__(self):
        status = "Disponível" if self.is_disponivel() else "Emprestado"
        return f"Livro: '{self.titulo}' | Autor: {self.__autor} | Status: {status}"

# --- CLASSE USUARIO (simples, sem encapsulamento para este exemplo) ---
class Usuario:
    def __init__(self, nome, id_usuario):
        self.nome = nome
        self.id = id_usuario
        self.livros_emprestados = [] # Lista de títulos

    def __str__(self):
        return f"Usuário ID: {self.id} - Nome: {self.nome}"

In [18]:
class Biblioteca:
    def __init__(self):
        self.acervo = {} # {titulo do livro: livro_objeto}
        self.usuarios = {} # {id_usuario: usuario_objeto}

    def adicionar_livro(self, livro):
        # NOTA: Com livros que representam cópias únicas (como neste modelo de encapsulamento),
        # você adicionaria apenas a primeira cópia, sem incrementar "copias_totais".
        # Manterei a lógica original do seu código anterior para evitar quebras.
        if livro.titulo in self.acervo:
            # Lógica para MÚLTIPLAS CÓPIAS (mantida do seu código)
            self.acervo[livro.titulo].copias_disponiveis += 1
            self.acervo[livro.titulo].copias_totais += 1
            print(f"✅ Adicionado(s) mais 1 cópia de '{livro.titulo}'.")
        else:
            # Lógica para CÓPIA ÚNICA (novo título)
            self.acervo[livro.titulo] = livro
            print(f"✅ Livro '{livro.titulo}' adicionado ao acervo.")

    def cadastrar_usuario(self, usuario):
        if usuario.id not in self.usuarios:
            self.usuarios[usuario.id] = usuario
            print(f"✅ Usuário '{usuario.nome}' (ID: {usuario.id}) cadastrado com sucesso.")
        else:
            print(f"❌ Erro: ID de usuário {usuario.id} já existe.")

    # --- Métodos de Operação (Empréstimo/Devolução) ---

    def emprestar_livro(self, id_usuario, titulo_livro):
        # 1. Verificar se o usuário existe
        if id_usuario not in self.usuarios:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return

        # 2. Verificar se o livro existe no acervo
        if titulo_livro not in self.acervo:
            print(f"❌ Erro: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        livro = self.acervo[titulo_livro]
        usuario = self.usuarios[id_usuario]

        # 3. Chamar o método ENCAPSULADO do objeto Livro
        if livro.emprestar():
            # A operação foi bem-sucedida (o Livro estava disponível)
            usuario.livros_emprestados.append(livro.titulo)
            print(f"🎉 Livro '{titulo_livro}' emprestado com sucesso para {usuario.nome}.")
        else:
            # O próprio Livro informa que não estava disponível
            print(f"⚠️ Livro '{titulo_livro}' está atualmente emprestado.")

    def devolver_livro(self, id_usuario, titulo_livro):
        # 1. Verificar se o usuário existe
        if id_usuario not in self.usuarios:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return

        usuario = self.usuarios[id_usuario]

        # 2. Verificar se o usuário realmente tem o livro emprestado
        if titulo_livro not in usuario.livros_emprestados:
            print(f"❌ Erro: O usuário {usuario.nome} não tem o livro '{titulo_livro}' emprestado.")
            return

        # 3. Realizar a devolução (remover do usuário)
        usuario.livros_emprestados.remove(titulo_livro)

        # 4. Chamar o método ENCAPSULADO do objeto Livro para marcar como disponível
        if titulo_livro in self.acervo:
            livro = self.acervo[titulo_livro]
            livro.devolver() # Método privado marca o livro como __disponivel = True
            print(f"🎉 Livro '{titulo_livro}' devolvido com sucesso por {usuario.nome}.")
        else:
             print(f"⚠️ Livro devolvido, mas não encontrado no acervo (aviso).")

    # --- Métodos de Visualização ---

    def listar_livros_disponiveis(self):
        print("\n--- Acervo Disponível ---")
        # Filtra usando o método encapsulado is_disponivel()
        disponiveis = [l for l in self.acervo.values() if l.is_disponivel()]
        if disponiveis:
            for livro in disponiveis:
                print(f"- {livro}")
        else:
            print("Nenhum livro disponível no momento.")

    def listar_emprestimos_usuario(self, id_usuario):
        if id_usuario in self.usuarios:
            usuario = self.usuarios[id_usuario]
            print(f"\n--- Livros Emprestados por {usuario.nome} (ID: {id_usuario}) ---")
            if usuario.livros_emprestados:
                for titulo in usuario.livros_emprestados:
                    print(f"- {titulo}")
            else:
                print("Nenhum livro emprestado.")
        else:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")

In [15]:
class Biblioteca:
    def __init__(self):
        self.acervo = {} # {titulo do livro: livro_objeto}
        self.usuarios = {} # {id_usuario: usuario_objeto}

    def adicionar_livro(self, livro): # Nome do método corrigido
        # Verifica se o livro (pelo título) já está no acervo
        if livro.titulo in self.acervo:
            # Se já exi ste, apenas aumenta o contador de cópias
            self.acervo[livro.titulo].copias_disponiveis += livro.copias_totais
            self.acervo[livro.titulo].copias_totais += livro.copias_totais
            print(f"✅ Adicionado(s) mais {livro.copias_totais} cópia(s) de '{livro.titulo}'.")
        else:
            # Se não existe, adiciona o livro (objeto) ao dicionário
            self.acervo[livro.titulo] = livro
            print(f"✅ Livro '{livro.titulo}' adicionado ao acervo.")

    def cadastrar_usuario(self, usuario):
        if usuario.id not in self.usuarios:
            self.usuarios[usuario.id] = usuario
            print(f"✅ Usuário '{usuario.nome}' (ID: {usuario.id}) cadastrado com sucesso.")
        else:
            print(f"❌ Erro: ID de usuário {usuario.id} já existe.")

    # --- Métodos de Operação (Empréstimo/Devolução) ---

    def emprestar_livro(self, id_

SyntaxError: incomplete input (ipython-input-3755656073.py, line 27)

In [7]:
class Livro:
    """
    Representa um livro com atributos privados e métodos de acesso (getters)
    e modificação (setters, como emprestar e devolver).
    """
    def __init__(self, titulo, autor):
        # Atributos "privados" (name mangling)
        self.__titulo = titulo
        self.__autor = autor
        self.__disponivel = True  # Começa sempre disponível para empréstimo

    # --- Getters (Métodos para Obter/Acessar os Atributos) ---

    def get_titulo(self):
        """Retorna o título do livro."""
        return self.__titulo

    def get_autor(self):
        """Retorna o autor do livro."""
        return self.__autor

    def is_disponivel(self):
        """Retorna o status de disponibilidade do livro (True/False)."""
        return self.__disponivel

    # --- Métodos de Comportamento (Modificam o estado privado) ---

    def emprestar(self):
        """
        Altera o status do livro para 'Indisponível' se ele estiver disponível.
        Retorna True se o empréstimo foi bem-sucedido, False caso contrário.
        """
        if self.__disponivel:
            self.__disponivel = False
            return True
        else:
            return False # O livro já estava emprestado

    def devolver(self):
        """
        Altera o status do livro para 'Disponível' (somente faz sentido
        se ele estiver indisponível).
        """
        self.__disponivel = True

    # --- Representação do Objeto ---

    def __str__(self):
        status = "Disponível" if self.__disponivel else "Emprestado"
        return f"Livro: '{self.__titulo}' | Autor: {self.__autor} | Status: {status}"

# ----------------------------------------------------------------------
# EXEMPLO DE USO
# ----------------------------------------------------------------------

livro1 = Livro("Padrões de Projeto", "Erich Gamma et al.")
livro2 = Livro("Código Limpo", "Robert C. Martin")

print(livro1) # Livro: 'Padrões de Projeto' | Autor: Erich Gamma et al. | Status: Disponível

# 1. Tentar Emprestar
print("\n--- Empréstimo ---")
if livro1.emprestar():
    print(f"✅ O livro '{livro1.get_titulo()}' foi emprestado.")
else:
    print("❌ Empréstimo não realizado.")

print(livro1) # Livro: 'Padrões de Projeto' | Autor: Erich Gamma et al. | Status: Emprestado

# 2. Tentar Emprestar Novamente (deve falhar)
print("\n--- Segundo Empréstimo ---")
if livro1.emprestar():
    print("✅ O livro foi emprestado novamente.")
else:
    print(f"⚠️ O livro '{livro1.get_titulo()}' JÁ está emprestado.")

# 3. Devolver
print("\n--- Devolução ---")
livro1.devolver()
print(f"✅ O livro '{livro1.get_titulo()}' foi devolvido.")

print(livro1) # Livro: 'Padrões de Projeto' | Autor: Erich Gamma et al. | Status: Disponível

Livro: 'Padrões de Projeto' | Autor: Erich Gamma et al. | Status: Disponível

--- Empréstimo ---
✅ O livro 'Padrões de Projeto' foi emprestado.
Livro: 'Padrões de Projeto' | Autor: Erich Gamma et al. | Status: Emprestado

--- Segundo Empréstimo ---
⚠️ O livro 'Padrões de Projeto' JÁ está emprestado.

--- Devolução ---
✅ O livro 'Padrões de Projeto' foi devolvido.
Livro: 'Padrões de Projeto' | Autor: Erich Gamma et al. | Status: Disponível


In [8]:
class Usuario:
  def __init__(self, nome, id_usuario):
    self.nome = nome
    self.id = id_usuario
    self.livros_emprestados = []
    def __str__(self):
      return f"Usuário ID: {self.id} Nome: {self.nome}"

In [13]:
class Biblioteca:
  def __init__(self):
    self.acervo = {} #[titulo do livro_objeto]
    self.usuarios = {} # id usuario para usuario_objeto

  def adcionar_livros(self, livro):
      if livro.titulo in self.acervo:
          self.acervo[livro.titulo].copias_disponiveis += livro.copias_totais
          self.acervo[livro.titulo].copias_totais += livro.copias_totais
          print(f"adcionado mais {livro.copias_totais} copias de {livro.titulo} ")
           else:
            self.acervo[livro.titulo] = livro
            print(f"✅ Livro '{livro.titulo}' adicionado ao acervo.")

  def cadastrar_usuario(self, usuario):
        if usuario.id not in self.usuarios:
            self.usuarios[usuario.id] = usuario
            print(f"✅ Usuário '{usuario.nome}' (ID: {usuario.id}) cadastrado com sucesso.")
        else:
            print(f"❌ Erro: ID de usuário {usuario.id} já existe.")

    # --- Métodos de Operação (Empréstimo/Devolução) ---

  def emprestar_livro(self, id_usuario, titulo_livro):
        # 1. Verificar se o usuário existe
        if id_usuario not in self.usuarios:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")
            return

        # 2. Verificar se o livro existe e tem cópias disponíveis
        if titulo_livro not in self.acervo:
            print(f"❌ Erro: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        livro = self.acervo[titulo_livro]

        if livro.copias_disponiveis > 0:
            # 3. Realizar o empréstimo
            livro.copias_disponiveis -= 1
            usuario = self.usuarios[id_usuario]
            usuario.livros_emprestados.append(livro.titulo)
            print(f"🎉 Livro '{titulo_livro}' emprestado com sucesso para {usuario.nome}.")
        else:
            print(f"⚠️ Livro '{titulo_livro}' não tem cópias disponíveis no momento.")

  def devolver_livro(self, id_usuario, titulo_livro):
        # 1. Verificar se o usuário existe
        if id_usuario not in self.usuarios:
            print(f" Erro: Usuário ID {id_usuario} não encontrado.")
            return

        usuario = self.usuarios[id_usuario]

        # 2. Verificar se o usuário realmente tem o livro emprestado
        if titulo_livro not in usuario.livros_emprestados:
            print(f" Erro: O usuário {usuario.nome} não tem o livro '{titulo_livro}' emprestado.")
            return

        # 3. Realizar a devolução
        usuario.livros_emprestados.remove(titulo_livro)

        # Aumentar a cópia disponível no acervo
        if titulo_livro in self.acervo:
            self.acervo[titulo_livro].copias_disponiveis += 1
            print(f"🎉 Livro '{titulo_livro}' devolvido com sucesso por {usuario.nome}.")
        else:
             # Isso só deve ocorrer se houver um erro de lógica, mas é bom prevenir
             print(f"⚠️ Livro devolvido, mas não encontrado no acervo (aviso).")

    # --- Métodos de Visualização ---

  def listar_livros_disponiveis(self):
        print("\n--- Acervo Disponível ---")
        disponiveis = [l for l in self.acervo.values() if l.copias_disponiveis > 0]
        if disponiveis:
            for livro in disponiveis:
                print(f"- {livro}")
        else:
            print("Nenhum livro disponível no momento.")

  def listar_emprestimos_usuario(self, id_usuario):
        if id_usuario in self.usuarios:
            usuario = self.usuarios[id_usuario]
            print(f"\n--- Livros Emprestados por {usuario.nome} (ID: {id_usuario}) ---")
            if usuario.livros_emprestados:
                for titulo in usuario.livros_emprestados:
                    print(f"- {titulo}")
            else:
                print("Nenhum livro emprestado.")
        else:
            print(f"❌ Erro: Usuário ID {id_usuario} não encontrado.")


IndentationError: unexpected indent (ipython-input-1214627434.py, line 11)